# Advanced features

In [1]:
# Add BipartitePandas to system path, do not run this
# import sys
# sys.path.append('../../..')

## Import the BipartitePandas package

Make sure to install it using `pip install bipartitepandas`.

In [2]:
import bipartitepandas as bpd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Get your data ready

For this notebook, we simulate data.

In [3]:
df = bpd.SimBipartite().simulate()
bdf = bpd.BipartiteDataFrame(i=df['i'], j=df['j'], y=df['y'], t=df['t'])
display(bdf)

,i,j,y,t
0,0,6,-1.295733,0
1,0,19,-0.988587,1
2,0,19,-2.672524,2
3,0,27,-1.317095,3
4,0,23,-1.169170,4
...,...,...,...,...
49995,9999,167,0.453329,0
49996,9999,96,1.694899,1
49997,9999,121,0.064668,2
49998,9999,121,1.988459,3


## Advanced data cleaning

*Hint:* want details on all cleaning parameters? Run `bpd.clean_params().describe_all()`, or search through `bpd.clean_params().keys()` for a particular key, and then run `bpd.clean_params().describe(key)`.

#### Compute the largest connected component

Use the parameter `connectedness` to set the connectedness measure to use, and `component_size_variable` to choose how to measure component size (e.g. `firms` chooses the connected component with the greatest number of unique firm ids, while `workers` chooses the connected component with the greatest number of unique worker ids).

*Note:* connectedness is NOT NECESSARILY maintained between non-collapsed and collapsed formats. Therefore, if you plan to use connected, collapsed data, it is recommended to first clean the non-collapsed data with `connectedness=None`, next collapse the data, and finally clean the collapsed data with the connectedness measure you plan to use.

#### Set how to handle worker-year duplicates

Use the parameter `i_t_how` to customize how worker-year duplicates are handled.

#### Collapse at the match-level

If you drop the `t` column, collapsing will automatically collapse at the match level. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

#### Avoid unnecessary copies

If you are working with a large dataset, you will want to avoid copies whenever possible. So set `copy=False`.

#### Avoid unnecessary sorts

If you know your data is sorted by `i` and `t` (or, if you aren't including a `t` column, just by `i`), then set `is_sorted=True`.

#### Avoid complicated loops

Sometimes workers leave a firm, then return to it (we call these workers *returners*). Returners can cause computational difficulties because sometimes intermediate firms get dropped (e.g. a worker goes from firm $A \to B \to A$, but firm $B$ gets dropped). This turns returners into stayers. This can change the largest connected set of firms, and if data is in collapsed format, requires the data to be recollapsed.

Because of these potential complications, if there are returners, many methods require loops that run until convergence.

These difficulties can be avoided by setting the parameter `drop_returns` (there are multiple ways to handle returners, they can be seen by running `bpd.clean_params().describe('drop_returns')`).

*Alternative:* another way to handle returners is to drop the `t` column. Then, sorting will automatically sort by `i` and `j`, which eliminates the possibility of returners. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

#### Disable logging

Logging can slow down data cleaning. Set the parameter `log=False` when constructing your dataframe to turn off logging.

#### Install Intel(R) Extension for Scikit-learn

Intel(R) Extension for Scikit-learn ([GitHub](https://github.com/intel/scikit-learn-intelex)) can speed up KMeans clustering.

## Simpler constructor

If the columns in your Pandas dataframe are already named correctly, you can simply put the dataframe as a parameter into the BipartitePandas dataframe constructor.

In [4]:
bdf = bpd.BipartiteDataFrame(df).clean()
display(bdf)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m,alpha,k,l,psi
0,0,6,-1.295733,0,1,-0.430727,0,1,-1.335178
1,0,19,-0.988587,1,1,-0.430727,0,1,-1.335178
2,0,19,-2.672524,2,1,-0.430727,0,1,-1.335178
3,0,27,-1.317095,3,2,-0.430727,1,1,-0.908458
4,0,23,-1.169170,4,1,-0.430727,1,1,-0.908458
...,...,...,...,...,...,...,...,...,...
49995,9999,167,0.453329,0,1,0.430727,8,3,0.908458
49996,9999,96,1.694899,1,2,0.430727,4,3,-0.114185
49997,9999,121,0.064668,2,1,0.430727,6,3,0.348756
49998,9999,121,1.988459,3,1,0.430727,6,3,0.348756


## Restoring original ids

To restore original ids, we need to make sure the dataframe is tracking ids as they change.

We make sure the dataframe tracks ids as they change by setting `include_id_reference_dict=True`.

Notice that in this example we use `j / 2`, so that `j` will be modified during data cleaning.

In [5]:
bdf_original_ids = bpd.BipartiteDataFrame(i=df['i'], j=(df['j'] / 2), y=df['y'], t=df['t'], include_id_reference_dict=True).clean()
display(bdf_original_ids)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m
0,0,0,-1.295733,0,1
1,0,1,-0.988587,1,1
2,0,1,-2.672524,2,1
3,0,2,-1.317095,3,2
4,0,3,-1.169170,4,1
...,...,...,...,...,...
49995,9999,87,0.453329,0,1
49996,9999,42,1.694899,1,2
49997,9999,188,0.064668,2,1
49998,9999,188,1.988459,3,1


#### Merging in original ids

The method `.original_ids()` will return a dataframe that merges in the original ids.

In [6]:
bdf_original_ids.original_ids()

,i,j,y,t,m,original_j
0,0,0,-1.295733,0,1,3.0
1,0,1,-0.988587,1,1,9.5
2,0,1,-2.672524,2,1,9.5
3,0,2,-1.317095,3,2,13.5
4,0,3,-1.169170,4,1,11.5
...,...,...,...,...,...,...
49995,9999,87,0.453329,0,1,83.5
49996,9999,42,1.694899,1,2,48.0
49997,9999,188,0.064668,2,1,60.5
49998,9999,188,1.988459,3,1,60.5


## Comparing dataframes

Dataframes can be compared using the utility method `bpd.util.compare_frames()`.

In [7]:
bpd.util.compare_frames(bdf, bdf.iloc[:len(bdf) // 2], size_variable='len', operator='geq')

True

## Filling in missing periods as unemployed

The method `.fill_missing_periods()` (for *Long* format) will fill in rows for missing intermediate periods. Note that this method will not work with custom columns.

*Hint:* filling in missing periods is a useful way to make sure collapsing data only collapses over worker-firm spells if they are for contiguous years.

In this example, we drop periods 1-3, then fill them in, setting `alpha` to become -1:

In [8]:
bdf_missing = bdf[(bdf['t'] == 0) | (bdf['t'] == 4)].clean()
display(bdf_missing.fill_missing_periods({'alpha': -1}))

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m,alpha,k,l,psi
0,0,6,-1.295733,0,1,-0.430727,0,1,-1.335178
1,0,-1,NaN,1,<NA>,-1.000000,<NA>,<NA>,NaN
2,0,-1,NaN,2,<NA>,-1.000000,<NA>,<NA>,NaN
3,0,-1,NaN,3,<NA>,-1.000000,<NA>,<NA>,NaN
4,0,23,-1.169170,4,1,-0.430727,1,1,-0.908458
...,...,...,...,...,...,...,...,...,...
49995,9999,167,0.453329,0,1,0.430727,8,3,0.908458
49996,9999,-1,NaN,1,<NA>,-1.000000,<NA>,<NA>,NaN
49997,9999,-1,NaN,2,<NA>,-1.000000,<NA>,<NA>,NaN
49998,9999,-1,NaN,3,<NA>,-1.000000,<NA>,<NA>,NaN


## Getting extended event study dataframes

BipartitePandas allows you to use *Long* format data to generate event studies with more than 2 periods.

You can specify:

- what column signals a transition (e.g. if `j` is used, a transition is when a worker moves firms)
- which column(s) should be treated as the event study outcome
- how many periods before and after the transition should be considered
- whether the pre- and/or post-trends must be stable, and for which column(s)

We consider an example where `j` is the transition column, `y` is the outcome column, and with pre- and post-trends of length 2 that are required to be at the same firm. Note that `y_f1` is the first observation after the individual moves firms.

In [9]:
es_extended = bdf.get_extended_eventstudy(transition_col='j', outcomes='y', periods_pre=2, periods_post=2, stable_pre='j', stable_post='j')
display(es_extended)

,i,t,y_l2,y_l1,y_f1,y_f2
0,6,3,-0.337792,-0.842950,2.286394,1.283994
1,7,2,-0.532382,0.415006,-1.368864,-1.714322
2,9,3,1.903130,0.555261,2.574896,0.139780
3,13,3,-1.584259,-1.931018,0.013608,0.499785
4,16,2,1.452526,-0.015363,0.575685,2.328791
...,...,...,...,...,...,...
2494,9966,2,1.354923,2.428121,2.081876,2.698139
2495,9985,2,2.578013,2.114894,2.129395,2.375844
2496,9988,2,1.164596,3.032755,0.828832,-0.128365
2497,9989,2,-0.022694,1.261927,-0.953840,-1.025316
